In [1]:
import sqlite3
from sqlite3 import OperationalError




In [2]:
def writeres(fn,res):
    f=open(fn,'w')
    f.write(str(res))
    f.close
    
    return open(fn).read()

In [ ]:
m=c.execute(' SELECT sql FROM sqlite_master')
m=c.execute('PRAGMA table_info( Frequency)')

In [ ]:
m.fetchall()

In [ ]:
#'σdocid=10398_txt_earn(frequency)'
m=c.execute('SELECT count(*) FROM Frequency WHERE DOCID="10398_txt_earn"')
f=open('part_a.txt','w')
f.write(str(m.fetchall()[0][0]))

In [ ]:
#πterm(σdocid=10398_txt_earn and count=1(frequency))
#part_b.txt
m=c.execute('SELECT count(*) FROM Frequency WHERE DOCID="10398_txt_earn" and COUNT=1')
f=open('part_b.txt','w')
f.write(str(m.fetchall()[0][0]))


In [ ]:
#πterm(σdocid=10398_txt_earn and count=1(frequency)) U πterm(σdocid=925_txt_trade and count=1(frequency))
m=c.execute('SELECT count(*) FROM  Frequency WHERE DOCID="10398_txt_earn" and COUNT=1 union SELECT count(*) FROM Frequency WHERE DOCID="925_txt_trade" and COUNT=1')
m=c.execute('SELECT * FROM  Frequency WHERE DOCID="10398_txt_earn" and COUNT=1 union all SELECT * FROM Frequency WHERE DOCID="925_txt_trade" and COUNT=1')

gt=m.fetchall()
len(gt)
#f=open('part_c.txt','w')
#f.write(str(sum(zip(*gt)[0])))

In [ ]:
query='''
SELECT count (*) From (
    SELECT F1.TERM FROM  Frequency F1
    WHERE F1.DOCID="10398_txt_earn" and F1.COUNT=1
    UNION
    SELECT F2.TERM FROM  Frequency F2
    WHERE F2.DOCID="925_txt_trade" and F2.COUNT=1
) x;
'''
res=c.execute(query).fetchall()
f=open('part_c.txt','w')
f.write(str(res[0][0]))
f.close
!cat part_c.txt

In [ ]:
res

In [ ]:
!cat part_c.txt

In [ ]:
#Write a SQL statement to count the number of unique documents
#containing the word "law" or containing the word "legal" 
#(If a document contains both law and legal, it should only be counted once)
conn = sqlite3.connect('reuters.db')
c = conn.cursor()

query='''
SELECT  COUNT(*) FROM
(SELECT DISTINCT F1.DOCID
    FROM Frequency F1
    WHERE F1.TERM LIKE 'legal' OR F1.TERM LIKE 'law')
;
'''
res=c.execute(query).fetchall()
c.close()
f=open('part_d.txt','w')
f.write(str(res[0][0]))
f.close
!cat part_d.txt


In [ ]:
#Write a SQL statement to find all documents that have more than 300 total terms,
#including duplicate terms. (Hint: You can use the HAVING clause, or you can use a nested query.
#Another hint: Remember that the count column contains the term frequencies, and you want to consider duplicates.)
#(docid, term_count)
fn='part_e.txt'
conn = sqlite3.connect('reuters.db')
c = conn.cursor()
query='''
    
    SELECT * FROM
    (SELECT F1.DOCID,SUBSTR(F1.DOCID,INSTR(F1.DOCID,'_')+5),F1.TERM,SUM(F1.COUNT)
    FROM Frequency F1
    GROUP BY F1.DOCID
    HAVING SUM(F1.COUNT)>300
    ORDER BY  SUBSTR(F1.DOCID,INSTR(F1.DOCID,'_')+5) ASC )
    
;
'''
query='''
    
    SELECT  sum(x.termcount) FROM
    (SELECT  F1.DOCID,SUM(F1.COUNT) as termcount
    FROM Frequency F1
    GROUP BY F1.DOCID
    HAVING COUNT(F1.DOCID)*SUM(F1.COUNT)>300
    )x
    
;
'''
query='select DISTINCT term || "," from (select (f.term)as term  from frequency f where f.docid="10000_txt_earn")'
query='''
select DISTINCT sum(x.c) from
(SELECT DISTINCT f.docid,count(f.docid) as c
FROM frequency f GROUP BY f.docid having sum(f.count)>300)x
'''
queryCorrect='''
select count(*) from 
(select distinct  f.docid,sum(f.term) 
from frequency f 
group by f.docid 
having sum(f.term)>300); 

'''
queryGrader='''
select count(*) from 
(select distinct  f.docid,count(f.term) 
from frequency f 
group by f.docid 
having count(f.term)>300); 

'''
res=c.execute(queryGrader).fetchall()
c.close()
writeres(fn,res[0][0])
!cat "$fn"
#sum(zip(*res)[1])
res

In [ ]:
''' two words: Write a SQL statement to count the 
number of unique documents that contain both the word 'transactions' and the word 'world'.
(Hint: Find the docs that contain one word and the 
docs that contain the other word separately, then find the intersection.)
'''
fn='part_f.txt'
conn = sqlite3.connect('reuters.db')
c = conn.cursor()

queryGrader='''
SELECT count(*) from (SELECT * 
FROM frequency f1 
WHERE f1.term ='transactions')x

INNER JOIN

(SELECT * 
FROM frequency f2
WHERE f2.term ='world')y

ON
x.docid=y.docid
'''
res=c.execute(queryGrader).fetchall()
c.close()
writeres(fn,res[0][0])
!cat "$fn"
#sum(zip(*res)[1])
res

In [ ]:
''' matrix product
'''
fn='part_g.txt'
conn = sqlite3.connect('matrix.db')
c = conn.cursor()

queryGrader='''
select res from
(SELECT a.row_num as r, b.col_num as c, SUM(a.value*b.value) as res
FROM a, b
WHERE a.col_num = b.row_num
GROUP BY a.row_num, b.col_num)x where x.r=2 and x.c=3 ;
'''
res=c.execute(queryGrader).fetchall()
c.close()
writeres(fn,res[0][0])
!cat "$fn"
#sum(zip(*res)[1])
res

In [23]:
''' similariy
'''
fn='part_h.txt'
conn = sqlite3.connect('reuters.db')
c = conn.cursor()

queryGrader='''
CREATE INDEX index_name
on FREQUENCY (DOCID, TERM);
'''
queryGrader='''
SELECT SUM(S) FROM
(SELECT F1T,F2T,SUM(Y.F2C*X.F1C) AS S FROM 
(SELECT F1.TERM AS F1T,F1.COUNT AS F1C FROM FREQUENCY F1 
    WHERE F1.DOCID='10080_txt_crude')X,

(SELECT F2.TERM AS F2T,F2.COUNT  AS F2C FROM FREQUENCY F2 
    WHERE F2.DOCID='17035_txt_earn')Y
WHERE Y.F2T=X.F1T
GROUP BY Y.F2T,X.F1T)
'''
#'''
#where x.r='10080_txt_crude' and x.c='17035_txt_earn';

#'''
res=c.execute(queryGrader).fetchall()
c.close()
writeres(fn,res[0][0])
!cat "$fn"
#sum(zip(*res)[1])
res

19

[(19,)]

#### keyword search: Find the best matching document to the keyword query 
"washington taxes treasury". 
You can add this set of keywords to the document corpus with a union of scalar queries:


In [98]:
''' similariy
'''
fn='part_i.txt'
conn = sqlite3.connect('reuters.db')
c = conn.cursor()

view='''
CREATE VIEW query AS
SELECT * FROM frequency
UNION
SELECT 'q' as docid, 'washington' as term, 1 as count 
UNION
SELECT 'q' as docid, 'taxes' as term, 1 as count
UNION 
SELECT 'q' as docid, 'treasury' as term, 1 as count

'''
queryGrader='''
SELECT MAX(c) FROM
(SELECT b.docid, b.term, SUM(a.count * b.count) as c
FROM query a, Frequency b
WHERE a.term = b.term 
AND a.docid = 'q'
GROUP BY b.docid, b.term
ORDER BY SUM(a.count * b.count) DESC);
'''



#'''
#where x.r='10080_txt_crude' and x.c='17035_txt_earn';

#'''
#res=c.execute(view)
res=c.execute(queryGrader).fetchall()
c.close()
#writeres(fn,res[0][0])
#!cat "$fn"
#sum(zip(*res)[1])
res

[(6,)]

In [36]:
sqllst=['SUM', 'FROM', 'SELECT', 'AS', 'WHERE','GROUP','BY','HAVING','ORDER']
queryGrader=queryGrader.lower()
for x in sqllst:
    if queryGrader.count(x.lower())> 0:
        queryGrader=queryGrader.replace(x.lower(),x)
print(queryGrader)


SELECT SUM(s) FROM
(SELECT f1t,f2t,SUM(y.f2c*x.f1c) AS s FROM 
(SELECT f1.term AS f1t,f1.count AS f1c FROM frequency f1 
    WHERE f1.docid='10080_txt_crude')x,

(SELECT f2.term AS f2t,f2.count  AS f2c FROM frequency f2 
    WHERE f2.docid='17035_txt_earn')y
WHERE y.f2t=x.f1t
GROUP BY y.f2t,x.f1t)



In [34]:
x=sqllst[0]
queryGrader=queryGrader.lower()
queryGrader.count(x.lower())
queryGrader.replace(x.lower(),x)

"\nselect SUM(s) from\n(select f1t,f2t,SUM(y.f2c*x.f1c) as s from \n(select f1.term as f1t,f1.count as f1c from frequency f1 \n    where f1.docid='10080_txt_crude')x,\n\n(select f2.term as f2t,f2.count  as f2c from frequency f2 \n    where f2.docid='17035_txt_earn')y\nwhere y.f2t=x.f1t\ngroup by y.f2t,x.f1t)\n"

In [ ]:
x